## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 5. Implementación de la Base de Vectores Final

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

In [2]:
from IPython.display import display, Markdown
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, pipeline
import re

### 1. Carga de Embeddings

In [5]:
# Se usa este modelo por ser ligero en cuestión de recursos computacionales
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
persist_path = "./database/embeddings_db"
# Especificación de la VectorDB
vector_store = SKLearnVectorStore(embedding=embeddings, persist_path=persist_path, serializer='parquet')

### 3. Carga del Modelo

In [ ]:
# Cargar un modelo de lenguaje preentrenado (Llama) para generación de texto
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map='auto'
)

In [ ]:
# Definir pipeline para la generación de texto
generate_text = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [ ]:
# Configurar el modelo de lenguaje dentro del pipeline de Hugging Face
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
llm = LlamaCpp(model_path="./llama-3.2-3b-instruct-q8_0.gguf", max_tokens=150)

In [32]:
vector_store= SKLearnVectorStore(
    embedding=embeddings, persist_path='./embeddings_db', serializer='parquet'
)

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1, "max_length": 300})

In [19]:
prompt_template = """
Utiliza la siguiente información para responder la pregunta al final de manera muy corta en un solo reglón.

Información:
{context}

Pregunta: {question}

Respuesta:
"""

In [15]:
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)

In [20]:
# Crear la cadena de preguntas y respuestas (QA) basada en el sistema RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
    }
)

query = "¿Qué limitaciones impone la transmisión de acciones de una ITF?"
result = qa_chain(query)

Llama.generate: 158 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    6760.09 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   150 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17478.60 ms /   151 tokens


In [21]:
display(Markdown(result['source_documents'][0].page_content))

conocimiento de ello. Cuando las adquisiciones y demás actos jurídicos a través de los cuales se obtenga directa o indirectamente la titularidad de acciones representativas del capital social de una ITF, se realicen sin obtener la autorización de la CNBV en contravención a lo d ispuesto en esta Ley o bien, existan indicios que señalen que los accionistas de las ITF dejaron de cumplir con los requisitos aplicables en términos

In [22]:
display(Markdown(result['result']))

La transmisión de acciones de una ITF está restringida por lo siguiente:
- Las transmisiones de acciones deben autorizarse previamente por la CNBV.
- La falta de autorización de la CNBV implica violación de los requisitos establecidos en la Ley, específicamente en el artículo que regula las transmisiones de valores. Por lo tanto, la transmisión de acciones sin la autorización previa de la CNBV constituye una violación del derecho y supone un incumplimiento de las obligaciones contractuales.

Debido a estas restricciones regulatorias, es esencial cumplir estrictamente con los requisitos legales antes de realizar cualquier